# NVDI PDP local Estimation

In [1]:
%pwd

'/Users/lichao/DP15_Article/12_PyCode'

In [2]:
%cd ..

/Users/lichao/DP15_Article


/Users/lichao/opt/anaconda3/envs/ML/lib/python3.9/site-packages/IPython/core/magics/osm.py:417: UserWarning: using dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


## Import Package 

In [3]:
from glob import glob
from joblib import dump, load
import joblib
import numpy as np
import pandas as pd
import random
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import train_test_split, KFold

## Load Data

### Connection NDVI

In [4]:
ori_df_list = []
pdp_df_list = []

for i in list(range(1, 11)):
    X_pdp = pd.read_parquet(f'03_Results/PdpLocal/PDP_Local_NDVI_{i}.parquet')
    length = int(X_pdp.shape[0]/41)
    print(length)
    ori_df_list.append(X_pdp.iloc[:length, :])
    
    y_ori_pred = X_pdp.iloc[:length, -1].to_list() * 41

    value_list = []
    for i in list(range(0, 41)):
        value_list = value_list + [0.05 * i] * length
    
    X_pdp['incre'] = value_list

    X_pdp['Pred_y_change'] = np.array(X_pdp['Pred_y']) - np.array(y_ori_pred)

    pdp_df_list.append(X_pdp)

38318
38318
38318
38317
38317
38317
38317
38317
38317
38317


In [5]:
pdp_df = pd.concat(pdp_df_list)
ori_df = pd.concat(ori_df_list)

In [6]:
pdp_df.shape

(15710093, 36)

In [7]:
ori_df.shape

(383173, 34)

### Build Local Connection

In [8]:
line_result_df = []
for line_row in range(ori_df.shape[0]):
    left = ori_df.iloc[line_row, 29]
    right = ori_df.iloc[line_row, 30]
    down = ori_df.iloc[line_row, 31]
    up = ori_df.iloc[line_row, 32]

    this_point_df = pdp_df[(pdp_df['lat']>down) & (pdp_df['lat']<up) & (pdp_df['lon']>left) & (pdp_df['lon']<right)]
    this_point_df = this_point_df[['incre', 'Pred_y_change']]

    pdp_local = this_point_df['Pred_y_change'].groupby(this_point_df['incre']).mean()
    #pdp_local = this_point_df
    pdp_local = pd.DataFrame(pdp_local).reset_index()
    
    X = pdp_local[['incre']]
    y = pdp_local['Pred_y_change']

    model = LinearRegression()
    model.fit(X, y)

    y_pred = model.predict(X)
    r2 = r2_score(y, y_pred)
    #print("R^2 Score:", r2, X.shape)

    # Standard Error of Coefficient
    n = len(y)
    residuals = y - y_pred
    residual_sum_of_squares = np.sum(residuals ** 2)
    x_var = np.sum((X['incre'] - X['incre'].mean()) ** 2)
    se_beta = np.sqrt(residual_sum_of_squares / (n - 2)) / np.sqrt(x_var) if n > 2 and x_var > 0 else np.nan

    
    lat = ori_df.iloc[line_row, 1]
    lon = ori_df.iloc[line_row, 2]

    line_result = [lat, lon, model.coef_[0], model.intercept_, r2, se_beta]

    if 

    line_result_df.append(line_result)

KeyboardInterrupt: 

In [ ]:
result_df = pd.DataFrame(line_result_df, columns = ['lat', 'lon', 'coeff', 'intercept', 'r2', 'SE_coeff'])

In [ ]:
result_df.to_excel('03_Results/PdpConnectionCoef/PdpConnection_NDVI_v2.xlsx')

### Connection NTL

In [ ]:
ori_df_list = []
pdp_df_list = []

for i in list(range(1, 11)):
    X_pdp = pd.read_parquet(f'03_Results/PdpLocal/PDP_Local_NTL_{i}.parquet')
    length = int(X_pdp.shape[0]/41)
    print(length)
    ori_df_list.append(X_pdp.iloc[:length, :])
    
    y_ori_pred = X_pdp.iloc[:length, -1].to_list() * 41

    value_list = []
    for i in list(range(0, 41)):
        value_list = value_list + [0.05 * i] * length
    
    X_pdp['incre'] = value_list

    X_pdp['Pred_y_change'] = np.array(X_pdp['Pred_y']) - np.array(y_ori_pred)

    pdp_df_list.append(X_pdp)

In [ ]:
pdp_df = pd.concat(pdp_df_list)
ori_df = pd.concat(ori_df_list)

In [ ]:
pdp_df.shape

In [ ]:
ori_df.shape

### Build Local Connection

In [ ]:
line_result_df = []
for line_row in range(ori_df.shape[0]):
    left = ori_df.iloc[line_row, 29]
    right = ori_df.iloc[line_row, 30]
    down = ori_df.iloc[line_row, 31]
    up = ori_df.iloc[line_row, 32]

    this_point_df = pdp_df[(pdp_df['lat']>down) & (pdp_df['lat']<up) & (pdp_df['lon']>left) & (pdp_df['lon']<right)]
    this_point_df = this_point_df[['incre', 'Pred_y_change']]

    pdp_local = this_point_df['Pred_y_change'].groupby(this_point_df['incre']).mean()
    #pdp_local = this_point_df
    pdp_local = pd.DataFrame(pdp_local).reset_index()
    
    X = pdp_local[['incre']]
    y = pdp_local['Pred_y_change']

    model = LinearRegression()
    model.fit(X, y)

    y_pred = model.predict(X)
    r2 = r2_score(y, y_pred)
    #print("R^2 Score:", r2, X.shape)

    # Standard Error of Coefficient
    n = len(y)
    residuals = y - y_pred
    residual_sum_of_squares = np.sum(residuals ** 2)
    x_var = np.sum((X['incre'] - X['incre'].mean()) ** 2)
    se_beta = np.sqrt(residual_sum_of_squares / (n - 2)) / np.sqrt(x_var) if n > 2 and x_var > 0 else np.nan
    
    lat = ori_df.iloc[line_row, 1]
    lon = ori_df.iloc[line_row, 2]

    line_result = [lat, lon, model.coef_[0], model.intercept_, r2, se_beta]

    line_result_df.append(line_result)

In [ ]:
result_df = pd.DataFrame(line_result_df, columns = ['lat', 'lon', 'coeff', 'intercept', 'r2', 'SE_coeff'])

In [ ]:
result_df.to_excel('03_Results/PdpConnectionCoef/PdpConnection_NTL_v2.xlsx')

### Connection income

In [ ]:
ori_df_list = []
pdp_df_list = []

for i in list(range(1, 11)):
    X_pdp = pd.read_parquet(f'03_Results/PdpLocal/PDP_Local_income_indiv_{i}.parquet')
    length = int(X_pdp.shape[0]/41)
    print(length)
    ori_df_list.append(X_pdp.iloc[:length, :])
    
    y_ori_pred = X_pdp.iloc[:length, -1].to_list() * 41

    value_list = []
    for i in list(range(0, 41)):
        value_list = value_list + [0.05 * i] * length
    
    X_pdp['incre'] = value_list

    X_pdp['Pred_y_change'] = np.array(X_pdp['Pred_y']) - np.array(y_ori_pred)

    pdp_df_list.append(X_pdp)

In [ ]:
pdp_df = pd.concat(pdp_df_list)
ori_df = pd.concat(ori_df_list)

In [ ]:
pdp_df.shape

In [ ]:
ori_df.shape

### Build Local Connection

In [ ]:
line_result_df = []
for line_row in range(ori_df.shape[0]):
    left = ori_df.iloc[line_row, 29]
    right = ori_df.iloc[line_row, 30]
    down = ori_df.iloc[line_row, 31]
    up = ori_df.iloc[line_row, 32]

    this_point_df = pdp_df[(pdp_df['lat']>down) & (pdp_df['lat']<up) & (pdp_df['lon']>left) & (pdp_df['lon']<right)]
    this_point_df = this_point_df[['incre', 'Pred_y_change']]

    pdp_local = this_point_df['Pred_y_change'].groupby(this_point_df['incre']).mean()
    #pdp_local = this_point_df
    pdp_local = pd.DataFrame(pdp_local).reset_index()
    
    X = pdp_local[['incre']]
    y = pdp_local['Pred_y_change']

    model = LinearRegression()
    model.fit(X, y)

    y_pred = model.predict(X)
    r2 = r2_score(y, y_pred)
    #print("R^2 Score:", r2, X.shape)

    # Standard Error of Coefficient
    n = len(y)
    residuals = y - y_pred
    residual_sum_of_squares = np.sum(residuals ** 2)
    x_var = np.sum((X['incre'] - X['incre'].mean()) ** 2)
    se_beta = np.sqrt(residual_sum_of_squares / (n - 2)) / np.sqrt(x_var) if n > 2 and x_var > 0 else np.nan
    
    lat = ori_df.iloc[line_row, 1]
    lon = ori_df.iloc[line_row, 2]

    line_result = [lat, lon, model.coef_[0], model.intercept_, r2, se_beta]

    line_result_df.append(line_result)

In [ ]:
result_df = pd.DataFrame(line_result_df, columns = ['lat', 'lon', 'coeff', 'intercept', 'r2', 'SE_coeff'])

In [ ]:
result_df.to_excel('03_Results/PdpConnectionCoef/PdpConnection_income_indiv_v2.xlsx')